#### 离散度的测量


离散度衡量的是一组数据的分布情况。它在金融领域尤为重要，因为衡量风险的主要方式之一，就是回溯历史上收益率的离散度。如果所有收益率都紧紧围绕在一个中心值左右，那么我们就不需要担心，反之如果收益率分散的到处都是，那就是有风险的。

低离散度的数据大量聚集在平均值附近，而高离散度的数据往往表示存在许多非常大和非常小的值（相较于平均值而言）。让我们通过生成一个随机整数数组来具体研究。

In [1]:
# 导入必要库
import numpy as np
# 设定随机数序列，这样每次运行生成的随机数都是固定的同一个，便于做分析
np.random.seed(121)

In [2]:
# 生成20个小于100的正整数
X = np.random.randint(100, size=20)

# 排序
X = np.sort(X)
print 'X: %s' %(X)

mu = np.mean(X)
print 'X的均值:', mu

X: [ 3  8 34 39 46 52 52 52 54 57 60 65 66 75 83 85 88 94 95 96]
X的均值: 60.2


#### 极差（又称范围误差或全距）

极差就是数据集中最大值减去最小值的差。 毫无疑问，它对（偏离均值的）异常值非常敏感。我们使用 `numpy`的峰对峰函数 (又称峰间函数）ptp来计算。

In [5]:
print 'X的极差: %s' %(np.ptp(X))

X的极差: 93


#### 平均绝对偏差 (MAD)

平均绝对偏差，是指各个观察值到算数平均值的平均距离，我们使用偏差的绝对值，以避免在计算平均偏差的过程中，高于均值的正数偏差与低于均值的负数偏差相互抵消的情况出现。平均绝对偏差公式为：

$$ MAD = \frac{\sum_{i=1}^n |X_i - \mu|}{n} $$

其中 $n$ 表示观察值的数量，$\mu$ 表示观察值的平均值.

In [6]:
abs_dispersion = [np.abs(mu - x) for x in X]
MAD = np.sum(abs_dispersion)/len(abs_dispersion)
print 'X的平均绝对偏差:', MAD

X的平均绝对偏差: 20.52


#### 方差和标准差

方差 $\sigma^2$ 定义为，围绕均值的所有偏差的平方和，取算数平均值:
$$ \sigma^2 = \frac{\sum_{i=1}^n (X_i - \mu)^2}{n} $$

方差有时候比平均绝对偏差更方便，因为绝对值函数（在x=0时）是不可导，然而方差是可导的，一些优化算法依赖于可导性。
标准差定义为方差的平方根， $\sigma$，它比方差更容易理解，因为它保持了与观察值相同的单位量纲（方差则对量纲进行了平方）。

In [7]:
print 'X的方差:', np.var(X)
print 'X的标准差:', np.std(X)

X的方差: 670.16
X的标准差: 25.8874486962


切比雪夫不等式是一种理解标准差的方法。它告诉我们，任意一个样本数据集中，位于其平均数m个标准差范围内的比例不小于$1 - 1/k^2$，其中$k$为大于1的任意正数。让我们来用之前的数据集检验一下。

In [10]:
k = 1.25
dist = k*np.std(X)
l = [x for x in X if abs(x - mu) <= dist]
print '位于平均值', k, '个标准差范围内的所有观察值:', l
print '确认', float(len(l))/len(X), '>', 1 - 1/k**2

位于平均值 1.25 个标准差范围内的所有观察值: [34, 39, 46, 52, 52, 52, 54, 57, 60, 65, 66, 75, 83, 85, 88]
确认 0.75 > 0.36


切比雪夫不等式给出的界限在这个例子中似乎相当宽松、并不严格, 当它很有用，以为它适用于所有数据集和分布。

#### 半方差和半偏差

尽管方差和标准差告诉我们一个“量”有多不稳定，但它们都不能区分出向上的偏差和向下的偏差。诸如像资产收益率，我们更关注向下的偏差。半方差和方差能够用来解决此类问题，只计算低于平均值的观测值。半方差的定义为：
$$ \frac{\sum_{X_i < \mu} (X_i - \mu)^2}{n_<} $$
其中 $n_<$ 表示小于均值的观察值个数。半方差的平方根即半偏差。

In [11]:
lows = [e for e in X if e <= mu]

semivar = np.sum( (lows - mu) ** 2 ) / len(lows)

print 'X的半方差:', semivar
print 'X的半偏差:', np.sqrt(semivar)

X的半方差: 689.512727273
X的半偏差: 26.2585743572


一个相关的概念是目标半方差(和目标半偏差)，它求得的是：所有低于目标值的观察值，距离该目标值的平均距离:

$$ \frac{\sum_{X_i < B} (X_i - B)^2}{n_{<B}} $$


In [12]:
B = 19
lows_B = [e for e in X if e <= B]
semivar_B = sum(map(lambda x: (x - B)**2,lows_B))/len(lows_B)

print 'X的目标半方差:', semivar_B
print 'X的目标半偏差:', np.sqrt(semivar_B)

X的目标半方差: 188
X的目标半偏差: 13.7113092008


#### 这些仅仅是估计
所有的这些计算会使你得到样本统计量，即样本数据的标准差。然而它是否能反映当前真实总体的标准差还不能断定，需要付出更多的努力来确认。这在金融领域中尤其成问题，因为所有的数据都是基于时间序列的，均值和方差可能随时间而变化。一般来说，不要因为你的样本是正确的就断言它在未来仍然是正确的。